# 7 - Boundary Layer Elements

In this tutorial, we discuss how we can add thin layers of prismatic elements to our meshes.

The motivation behind this is that in a highly conducting magnetic object, such as steel, the electromagnetic fields decay rapidly measured by its skin depth (the depth at which the field has decayed to $1/e$ of its surface value). For such an object there are very high field gradients close to the surface of the object.

These gradients can be captured by $h$ or $p$-refinement, however to save computational time and expense, we instead introduce thin layers of prismatic elements at the surface of the object. Combined with $p$-refinement this allows the skin depth effects to be captured accurately and allow us to accurately characterise highly magnetic objects.

<b> James : update to use material name rather than object names </b>

## Cube Example
Taking the cube example that we worked through in tutorial [8](./8_Generating_mesh_from_step_file.ipynb), we want to generate a homogeneous magnetic cube with a single thin layer of prismatic boundary layer elements of thickness 0.5 units.

<b>We do not need prismatic boundary layer elements for non-magnetic objects</b>

In this particular case, we consider a 10 mm cube of conductivity $\sigma_* = 10^6$ S/m and relative permeability $\mu_r = 100$. 

In [1]:
from netgen.occ import *

object_name = 'cube'
sigma = 1e6
mur = 100

geo = OCCGeometry(r'Examples/Example_10mm_cube.step')
cube = geo.shape.Move((-geo.shape.center.x, -geo.shape.center.y, -geo.shape.center.z))

cube.bc('default')
cube.mat(object_name)
cube.name = object_name
cube.faces.name = object_name
cube.maxh = 1

box = Box(Pnt(-1000, -1000, -1000), Pnt(1000,1000,1000))
box.mat('air')
box.bc('outer')
box.maxh=1000

joined_object = Glue([box, cube])
nmesh = OCCGeometry(joined_object).GenerateMesh(meshsize.coarse)

generates a standard tetrahedral mesh for a cube.

A thin layer of prismatic elements can be added to the surface of the cube via the $\texttt{BoundaryLayer}$ 
method. 

Using the $\texttt{help}$ function, we see the optional arguments for the $\texttt{BoundaryLayer}$ method.

In [2]:
help(nmesh.BoundaryLayer)

Help on method BoundaryLayer in module netgen.libngpy._meshing:

BoundaryLayer(...) method of netgen.libngpy._meshing.Mesh instance
    BoundaryLayer(self: netgen.libngpy._meshing.Mesh, boundary: Union[str, int], thickness: Union[float, list], material: str, domains: Union[str, int] = '.*', outside: bool = False, project_boundaries: Optional[str] = None, grow_edges: bool = True, limit_growth_vectors: bool = True) -> None
    
    
    Add boundary layer to mesh.
    
    Parameters
    ----------
    
    boundary : string or int
      Boundary name or number.
    
    thickness : float or List[float]
      Thickness of boundary layer(s).
    
    material : str or List[str]
      Material name of boundary layer(s).
    
    domain : str or int
      Regexp for domain boundarylayer is going into.
    
    outside : bool = False
      If true add the layer on the outside
    
    grow_edges : bool = False
      Grow boundary layer over edges.
    
    project_boundaries : Optional[str] 

We want to introduce a single thin layer of prisms to the interior of the cube so we need to set $\texttt{thickness=[0.5]}$, and $\texttt{domain='cube'}$.
We also want a homogeneous cube, so we also need to specify that the material of the prisms is the same as the material for the cube.

The final syntax we need is therefore

In [3]:
nmesh.BoundaryLayer(boundary=".*", thickness=[0.5], material=object_name,
                           domains=object_name, outside=False)

In [4]:
nmesh.Save('../VolFiles/OCC_cube_prism.vol')

If we wanted multiple layers of prismatic elements, we would specify a list of thicknesses. E.g. $\texttt{thickness=[0.5,0.3,0.2]}$.

 In Netgen, we can view the additional prismatic boundary layer via $\texttt{view} \rightarrow \texttt{Viewing Options} \rightarrow \texttt{Mesh} \rightarrow \texttt{Show Prisms}$ 
 
 <img src="Figures/cube_prism_example.jpg" alt="isolated" width="400"/>
 
This figure shows a cut though of the resultant mesh, $\texttt{OCC_cube_prism.vol}$, showing the tetrahedral mesh inside the cube in green, the non-conducting region in red, and the thin layer of prisms in cyan.

## Sphere Example
The $\texttt{BoundaryLayer}$ method requires a defined edge that it can use to correctly introduce the layer of prisms, such as the edge of a cube.

In the case of a sphere, no such edge exists, thus we need to introduce one.

We begin by defining a unit radius sphere with a conductivity $\sigma=10^6$ S/m and relative permeability $\mu_r = 80$

In [7]:
object_name = 'sphere'
sigma = 1e6
mur = 80

sphere = Sphere(Pnt(0,0,0), r=1)

Now we want to split the sphere into two hemispheres and rejoin them.

In [8]:
pos_sphere = sphere - Box(Pnt(0,100,100), Pnt(-100,-100,-100))
neg_sphere = sphere - Box(Pnt(0,100,100), Pnt(100,-100,-100))
sphere = pos_sphere + neg_sphere

If we now draw the sphere geometry, we see that we have introduced a bisecting plane

In [9]:
from netgen.webgui import Draw as DrawGeo
DrawGeo(sphere)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [0.0, 0.0, 0.0], 'mesh_radi…

BaseWebGuiScene

and now we can proceed as before by defining a outer region, and assigning material names ect

In [10]:
box = Box(Pnt(-1000,-1000,-1000), Pnt(1000,1000,1000))
box.bc('outer')
box.mat('air')

sphere.mat(object_name)
sphere.bc('default')
sphere.name = object_name
sphere.faces.name = object_name
sphere.maxh = 0.2

and generate a coarse mesh

In [11]:
joined_object = Glue([box, sphere])
nmesh = OCCGeometry(joined_object).GenerateMesh(meshsize.coarse)

In this example, we wish to introduce three layers of prismatic elements of thicknesses 0.001 0.005, and 0.05 units respectivly. We therefore call $\texttt{BoundaryLayer}$ as

In [12]:
nmesh.BoundaryLayer(boundary=".*",thickness=[1e-3, 5e-3, 5e-2], material=object_name,
                   domains=object_name, outside=False)

The equivalent $\texttt{.py}$ file for this example can be found in [$\texttt{OCC_Geometry/OCC_sphere_prism.py}$](../OCC_Geometry/OCC_sphere_prism.py).

## $p$ Refinement Convergence 

With the introduction of the prismatic layer, we see that MPT-Calculator is able to acheive converged results over a much wider frequency range than the equivalent tetrahedral mesh. 

Considering the sphere defined above with $\mu_r=80$, $\sigma=10^6$, and $\alpha=0.01$ m, we go from
<table><tr>
    <td><img src="Figures/Real_sphere_mur=80.png" alt="isolated" width="400"/></td>  
    <td><img src="Figures/Imag_sphere_mur=80.png" alt="isolated" width="400"/></td>
</tr></table>
 
where there are no prismatic elements to
 
<table><tr>
    <td><img src="Figures/Real_prism_sphere_mur=80.png" alt="isolated" width="400"/></td>  
    <td><img src="Figures/Imag_prism_sphere_mur=80.png" alt="isolated" width="400"/></td>
</tr></table>
 where the three layers of prismatic elements have been included, and see that the solutions converge across the entire frequency range using $p=2$ order elements.
 
 [1] J. Elgy, P. D. Ledger, J. L. Davidson, T. Özdeğer, A.J. Peyton, "Computation and measurement of magnetic polarizability tensor object characterisation of highly conducting and permeable objects, in preparation